In [1]:
import json
import random
random.seed(42)
# Load the list of dictionaries from a JSON file
with open('/workspace/CS762_Project/Datasets/Filtered/combined-Filtered.json', 'r') as json_file:
    data = json.load(json_file)
# random.shuffle(data)

In [19]:
random.seed(42)
def split_for_data_quantity(data):
    data_splits = {}
    for i in range(9,18):
        sample_size = 2**i
        data_splits[sample_size] = random.sample(data, sample_size)
    return data_splits

In [20]:
random.seed(42)
data_splits = split_for_data_quantity(data)
with open('/workspace/CS762_Project/Ablations/Data_Quantity/data_splits_powers_of_2.json', 'w') as file:
    json.dump(data_splits, file)

In [21]:
{d:len(data_splits[d]) for d in data_splits}

{512: 512,
 1024: 1024,
 2048: 2048,
 4096: 4096,
 8192: 8192,
 16384: 16384,
 32768: 32768,
 65536: 65536,
 131072: 131072}

In [29]:
random.shuffle(data)
for idx,d in enumerate(data[:1000]):
    print(f'Sample {idx}')
    print('Question:')
    print(d['question'])
    print('Code:')
    print(d['question'])
    print()

Sample 0
Read NOAA USCRN [1]_ [2]_ fixed-width file into pandas dataframe.

    Parameters
    ----------
    filename: str
        filepath or url to read for the fixed-width file.

    Returns
    -------
    data: Dataframe
        A dataframe with DatetimeIndex and all of the variables in the
        file.

    Notes
    -----
    CRN files contain 5 minute averages labeled by the interval ending
    time. Here, missing data is flagged as NaN, rather than the lowest
    possible integer for a field (e.g. -999 or -99). Air temperature in
    deg C. Wind speed in m/s at a height of 1.5 m above ground level.

    Variables corresponding to standard pvlib variables are renamed,
    e.g. `SOLAR_RADIATION` becomes `ghi`. See the
    `pvlib.iotools.crn.VARIABLE_MAP` dict for the complete mapping.

    References
    ----------
    .. [1] U.S. Climate Reference Network
       `https://www.ncdc.noaa.gov/crn/qcdatasets.html
       <https://www.ncdc.noaa.gov/crn/qcdatasets.html>`_

    .. [2]

In [3]:
import evaluate
perplexity = evaluate.load("perplexity", module_type="metric")
input_texts = []
random.shuffle(data)
for i in data[:100]:
    temp_str = f"Question:\n{i['question']}\nCode:\n{i['code']}"
    input_texts.append(temp_str)
print([len(t) for t in input_texts])
input_texts = [t for t in input_texts if len(t)<500]
results = perplexity.compute(model_id='/workspace/CS762_Project/CodeLlama-7b-Python-hf',
                             add_start_token=False,
                             predictions=input_texts)
for i,j in zip(input_texts, results['perplexities']):
    print(f'Input text:\n{i}\nResult:\n{j}')

[1823, 398, 600, 952, 2699, 326, 492, 2246, 1225, 1538, 479, 1668, 1361, 1081, 283, 443, 336, 1318, 263, 393, 532, 291, 555, 1086, 302, 1347, 1240, 531, 329, 483, 1120, 699, 1365, 562, 580, 1151, 3977, 702, 810, 518, 852, 1223, 262, 226, 1688, 472, 779, 567, 1085, 450, 1490, 443, 2057, 476, 429, 1335, 919, 228, 741, 2225, 179, 331, 578, 2568, 449, 575, 455, 342, 608, 474, 907, 288, 976, 1141, 522, 452, 226, 435, 1529, 447, 1006, 208, 583, 497, 926, 548, 1211, 707, 964, 868, 1469, 1162, 540, 696, 560, 2000, 476, 410, 570, 626]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Input text:
Question:
get RtlNetlist context from signals
Code:
def _get_rtl_context(self):
        
        for sig in chain(self._inputs, self._outputs):
            if sig.ctx:
                return sig.ctx
            else:
                # Param instances does not have context
                continue
        raise HwtSyntaxError(
            "Statement does not have any signal in any context", self)
Result:
8.809654235839844
Input text:
Question:
Design a python program that calculates a person's BMI (body mass index).
Code:

def calculate_bmi():
    weight = float(input('Enter your weight in kilograms: '))
    height = float(input('Enter your height in meters: '))
    bmi = weight / (height ** 2)
    print(f'Your BMI is: {round(bmi, 2)}')

calculate_bmi()

Result:
2.3518307209014893
Input text:
Question:
Write a physical package (.pptx file) to *pkg_file* containing
        *pkg_rels* and *parts* and a content types stream based on the
        content types of the parts.
Code:

In [2]:
p1 = "Develop a function that accepts a numerical array and additional singular integer items to be added as a tail-end element."
p2 = "Develop a program to evaluate if the provided mathematical expression is valid or not. The program should check for correct usage of parentheses, brackets, and braces, as well as proper placement of operators. Additionally, the program should also validate the usage of mathematical functions like sin, cos, tan, log, sqrt, etc. and constants like pi, e, etc. The program should also be able to handle nested functions and operators. The mathematical expression can be in infix, prefix, or postfix notation."
p3 = """Release a semaphore, incrementing the internal counter by one.

        When the counter is zero on entry and another thread is waiting for it
        to become larger than zero again, wake up that thread.

        If the number of releases exceeds the number of acquires,
        raise a ValueError."""
p4 = "Given a list l=[1,2,6,4,9,0,3,4,10], sort this list in ascending order using python"
p5 = 'reverse a string in python'
input_texts = [p1, p2, p3,p4,p5]

In [39]:
results

{'perplexities': [85.74868774414062,
  4.956254959106445,
  2.8436410427093506,
  9.028696060180664,
  211.1890411376953],
 'mean_perplexity': 62.75326418876648}

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import evaluate
perplexity = evaluate.load("perplexity", module_type="metric")
checkpoint = '/workspace/CS762_Project/CodeLlama-7b-Python-hf'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)  # You may want to use bfloat16 and/or move to GPU here
tokenized_chats=[]
for i in input_texts:
    messages = [
        {
            "role": "system",
            "content": "Given a question relating to solving programming problems, you must answer the question appropriately",
        },
        {"role": "user", "content": i},
     ]
    
    tokenized_chats.append(tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"))
# print(tokenized_chats)
# results = perplexity.compute(model_id='/workspace/CS762_Project/CodeLlama-7b-Python-hf',
#                          add_start_token=False,
#                          predictions=tokenized_chats)
# print(results)
    # print(tokenizer.decode(tokenized_chat[0]))
    # outputs = model.generate(tokenized_chat, max_new_tokens=1024) 
    # print(tokenizer.decode(outputs[0]))

In [12]:
tokenized_chats

[tensor([[    1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13, 29954,
           5428,   263,  1139,  1104,  1218,   304, 17069,  8720,  4828, 29892,
            366,  1818,  1234,   278,  1139,  7128,  2486,    13, 29966,   829,
          14816, 29903,  6778,    13,    13, 21956,   263,   740,   393, 21486,
            263, 16259,  1409,   322,  5684, 13512,  6043,  4452,   304,   367,
           2715,   408,   263, 12464, 29899,   355,  1543, 29889,   518, 29914,
          25580, 29962]]),
 tensor([[    1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13, 29954,
           5428,   263,  1139,  1104,  1218,   304, 17069,  8720,  4828, 29892,
            366,  1818,  1234,   278,  1139,  7128,  2486,    13, 29966,   829,
          14816, 29903,  6778,    13,    13, 21956,   263,  1824,   304, 14707,
            565,   278,  4944, 19475,  4603,   338,  2854,   470,   451, 29889,
            450,  1824,   881,  1423,   363,  1959,  8744,   310, 29494, 29892,
          204

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the pretrained model and tokenizer
model_name = '/workspace/CS762_Project/CodeLlama-7b-Python-hf'  # Change this to the model you want to use
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

p1 = "Develop a function that accepts a numerical array and additional singular integer items to be added as a tail-end element."
p2 = "Develop a program to evaluate if the provided mathematical expression is valid or not. The program should check for correct usage of parentheses, brackets, and braces, as well as proper placement of operators. Additionally, the program should also validate the usage of mathematical functions like sin, cos, tan, log, sqrt, etc. and constants like pi, e, etc. The program should also be able to handle nested functions and operators. The mathematical expression can be in infix, prefix, or postfix notation."
p3 = """Release a semaphore, incrementing the internal counter by one.

        When the counter is zero on entry and another thread is waiting for it
        to become larger than zero again, wake up that thread.

        If the number of releases exceeds the number of acquires,
        raise a ValueError."""
p4 = "Given a list l=[1,2,6,4,9,0,3,4,10], sort this list in ascending order using python"
p5 = 'reverse a string in python'
input_texts = [p1, p2, p3,p4,p5]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
import torch
for i in tokenized_chats:
    generation = model(i)
    logits = generation.logits[0]
    print(type(logits))
    print(len(logits))
    print(f'Perplexity: {torch.exp(logits).item()}')

KeyboardInterrupt: 

In [23]:
torch.exp(logits).item()

RuntimeError: a Tensor with 1984000 elements cannot be converted to Scalar

In [25]:


# Initialize a list to store perplexity values
perplexities = []

# Loop through the input texts
for input_text in tokenized_chats:
    # Tokenize the input text
    # input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # print(input_ids)
    # Generate output text from the model
    output = model.generate(input_text, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

    # Decode the generated output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Tokenize the generated output
    generated_tokens = tokenizer.encode(generated_text, return_tensors="pt")
    print(generated_tokens.shape)
    # Calculate the perplexity of the generated output
    with torch.no_grad():
        logits = model(input_ids).logits

    # Calculate the loss with respect to the logits
    loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.shape[-1]), generated_tokens.view(-1))
    
    # Calculate the perplexity of the generated output
    perplexity = torch.exp(loss).item()

    perplexities.append(perplexity)

# Print the generated texts and perplexity values for each input
for input_text, perplexity in zip(input_texts, perplexities):
    print(f"Input Text: {input_text}")
    print(f"Generated Text: {generated_text}")
    print(f"Perplexity: {perplexity}")
    print("\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 62, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


torch.Size([1, 63])


ValueError: Expected input batch_size (24) to match target batch_size (63).

In [34]:
model_name = '/workspace/CS762_Project/CodeLlama-7b-Python-hf'  # Change this to the model you want to use
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
p1 = "Develop a function that accepts a numerical array and additional singular integer items to be added as a tail-end element."
p2 = "Develop a program to evaluate if the provided mathematical expression is valid or not. The program should check for correct usage of parentheses, brackets, and braces, as well as proper placement of operators. Additionally, the program should also validate the usage of mathematical functions like sin, cos, tan, log, sqrt, etc. and constants like pi, e, etc. The program should also be able to handle nested functions and operators. The mathematical expression can be in infix, prefix, or postfix notation."
p3 = """Release a semaphore, incrementing the internal counter by one.

        When the counter is zero on entry and another thread is waiting for it
        to become larger than zero again, wake up that thread.

        If the number of releases exceeds the number of acquires,
        raise a ValueError."""
p4 = "Given a list l=[1,2,6,4,9,0,3,4,10], sort this list in ascending order using python"
p5 = 'reverse a string in python'
p6 = "Given a list of probabilities, find the item in the list that is the most probable and multiply it by 1000"
input_texts = [p1, p2, p3,p4,p5, p6]

def map_indices_to_tokens(tensor, tokenizer):
    # Find the argmax of each row
    argmax_indices = torch.argmax(tensor, dim=2)
    
    # Map each index to its corresponding token
    tokens = [tokenizer.decode(idx) for idx in argmax_indices[0]]

    return ''.join(tokens)


def calculate_perplexity(text, model, tokenizer):
    # Encode the text and add batch dimension
    inputs = tokenizer(text, return_tensors='pt')
    # Move input to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Calculate log likelihood
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        output_str = map_indices_to_tokens(outputs.logits, tokenizer)
        print(output_str)
        log_likelihood = outputs[0] * -1
    print(log_likelihood)
    # Calculate perplexity
    perplexity = torch.exp(log_likelihood / inputs["input_ids"].shape[1])
    return perplexity.item()

for text in input_texts:
    messages = [
        {
            "role": "system",
            "content": "Given a question relating to solving programming problems, you must answer the question appropriately",
        },
        {"role": "user", "content": text},
     ]
    template_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, return_tensors="pt")
    print(template_text)
    perplexity = calculate_perplexity(template_text, model, tokenizer)
    print(f"Perplexity of '{text}': {perplexity}")


<s>[INST] <<SYS>>
Given a question relating to solving programming problems, you must answer the question appropriately
<</SYS>>

Develop a function that accepts a numerical array and additional singular integer items to be added as a tail-end element. [/INST]
importimport![ALL
<N_

PIalist,atingtoaaproblems,Iareansweritquestion.ately.
br<S>>

[edprogramthattakesastringvalueandreturnsparametersintegervalue.beaddedtoparametersparametertoendof.
1]]

tensor(-4.1791)
Perplexity of 'Develop a function that accepts a numerical array and additional singular integer items to be added as a tail-end element.': 0.9358166456222534
<s>[INST] <<SYS>>
Given a question relating to solving programming problems, you must answer the question appropriately
<</SYS>>

Develop a program to evaluate if the provided mathematical expression is valid or not. The program should check for correct usage of parentheses, brackets, and braces, as well as proper placement of operators. Additionally, the program should 